<a href="https://colab.research.google.com/github/NilayGaitonde/stackoverflow-questions/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import shutil
import string

import numpy as np
import matplotlib.pyplot as plt
import re

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download("punkt")
nltk.download("stopwords")
stopwords_english = set(stopwords.words("english"))
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, RNN, Embedding, Activation, Dropout, GlobalAveragePooling1D, Flatten, TextVectorization, LSTM, GRU
from tensorflow.keras.losses import CategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy, BinaryAccuracy

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


unzip:  cannot find or open /usr/share/nltk_data/corpora/wordnet.zip, /usr/share/nltk_data/corpora/wordnet.zip.zip or /usr/share/nltk_data/corpora/wordnet.zip.ZIP.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
url = "https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz"

dataset = keras.utils.get_file(fname="stackoverflow",origin=url,cache_dir=".",cache_subdir="",untar=True)


6053168/6053168 [==============================] - 0s 0us/step


In [ ]:
with open("test/csharp/1.txt","r") as f:
    print(f.read())

"maxdegreeofparallelism not limiting the number of parallel tasks i am creating a number of threads and want to execute only n at a time. but i have a problem that when the code is executed it starts all threads at once...for (int i = 0; i &lt; global.mylist.count - 1; i++).{.    thread thread = new thread(() =&gt;.    {.        debug.writeline(""start signal from thread"", thread.currentthread.name);.        var account = global.mylist[i];.        thread.sleep(5000);.        debug.writeline(""end signal from thread"", thread.currentthread.name);.    });.    thread.name = i.tostring();.    threads.add(thread);.}..var option = new paralleloptions().{.    maxdegreeofparallelism = convert.toint32(numthreads.value).};..var locker = new object();.parallel.foreach(threads, option, t =&gt;.{.    if (t.isalive == false).    {.        t.start();.    }.    var count = convert.toint32(t.name);.    interlocked.increment(ref count);.    lock (locker).    {.        debug.writeline(""number of active

In [ ]:
raw_test_ds = keras.utils.text_dataset_from_directory("/content/test",batch_size=50,validation_split=0.2,subset="training",label_mode="categorical",seed=42)
raw_val_ds = keras.utils.text_dataset_from_directory("/content/train",batch_size=50,validation_split=0.2,subset="validation",label_mode="categorical",seed=42)
raw_train_ds = keras.utils.text_dataset_from_directory("/content/train",batch_size=50,label_mode="categorical")


Found 8000 files belonging to 4 classes.
Using 6400 files for training.
Found 8000 files belonging to 4 classes.
Using 1600 files for validation.
Found 8000 files belonging to 4 classes.


In [ ]:
for text,label in raw_train_ds.take(1):
    print(text,label)

tf.Tensor(
[b'"blank ternary operator i am creating two different icons for a google map. .however only one icon appears..a drop down select called facility will allow one icon based on a value of \'8\', all other values create the other icon...var marker = createmarker(latlng, name, address, city, state, zipcode, telephone, images, url, facility);...function createmarker(latlng, name, address, city, state, zipcode, telephone, images, url, facility) { .  var html = ""&lt;div id=\'infodiv\'&gt;"" + name + ""&lt;br/&gt;&lt;h4&gt;"" + address + ""&lt;br/&gt;"" + city + "", "" + state + "" "" + zipcode + ""&lt;br/&gt;"" + (formatphone(telephone)) + ""&lt;/h4&gt;&lt;div class=\'infoimage\'&gt;&lt;img src=\'"" + images + ""\'&gt;&lt;/div&gt;&lt;div class=\'footer\'&gt;&lt;a href=\'http://"" + url + ""\'&gt;"" + url + ""&lt;/a&gt;&lt;/div&gt;&lt;/div&gt;"";.  var marker = new google.maps.marker({.    icon: icon2,.    icon: (facility == 8) ? icon : icon, .    map: map, .    position: latlng . 

In [ ]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

In [ ]:
max_features = 10000
sequence_length = 1000

vectorize_layer = TextVectorization(max_tokens=max_features,standardize=custom_standardization,output_sequence_length=sequence_length,output_mode="int")

In [ ]:
train_text = raw_train_ds.map(lambda x, y: x)
# type(train_text)
vectorize_layer.adapt(train_text)

In [ ]:
len(vectorize_layer.get_vocabulary())

10000

In [ ]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label
train_ds = raw_train_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size = AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size = AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size = AUTOTUNE)


In [ ]:
model = Sequential([
    Embedding(max_features,20),
    Dropout(0.2),
    GlobalAveragePooling1D(),
    Dropout(0.2),
    Dense(4,activation='sigmoid')
])

model.compile(loss=CategoricalCrossentropy(),metrics=CategoricalAccuracy(),optimizer="adam")
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, None, 20)          200000    
                                                                 
 dropout_13 (Dropout)        (None, None, 20)          0         
                                                                 
 global_average_pooling1d_7  (None, 20)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dropout_14 (Dropout)        (None, 20)                0         
                                                                 
 dense_7 (Dense)             (None, 4)                 84        
                                                                 
Total params: 200084 (781.58 KB)
Trainable params: 200084 (781.58 KB)
Non-trainable params: 0 (0.00 Byte)
______________

In [ ]:
history = model.fit(train_ds,validation_data=val_ds,epochs=100,verbose=1)

Epoch 1/100
160/160 [==============================] - 4s 25ms/step - loss: 1.1281 - categorical_accuracy: 0.6570 - val_loss: 1.0941 - val_categorical_accuracy: 0.6888
Epoch 2/100
160/160 [==============================] - 5s 28ms/step - loss: 1.0906 - categorical_accuracy: 0.6739 - val_loss: 1.0572 - val_categorical_accuracy: 0.7075
Epoch 3/100
160/160 [==============================] - 3s 22ms/step - loss: 1.0568 - categorical_accuracy: 0.6920 - val_loss: 1.0202 - val_categorical_accuracy: 0.7225
Epoch 4/100
160/160 [==============================] - 4s 22ms/step - loss: 1.0237 - categorical_accuracy: 0.7065 - val_loss: 0.9848 - val_categorical_accuracy: 0.7406
Epoch 5/100
160/160 [==============================] - 5s 29ms/step - loss: 0.9909 - categorical_accuracy: 0.7216 - val_loss: 0.9516 - val_categorical_accuracy: 0.7469
Epoch 6/100
160/160 [==============================] - 4s 22ms/step - loss: 0.9606 - categorical_accuracy: 0.7285 - val_loss: 0.9195 - val_categorical_accuracy:

In [ ]:
loss, accuracy = model.evaluate(test_ds)
print(f"Loss: {loss}\nAccuracy: {accuracy}")

128/128 [==============================] - 2s 18ms/step - loss: 0.5343 - categorical_accuracy: 0.8127
Loss: 0.5343488454818726
Accuracy: 0.8126562237739563


In [ ]:
export_model = tf.keras.Sequential([
  vectorize_layer,
  model,
])

export_model.compile(
    loss=CategoricalCrossentropy(), optimizer="adam", metrics=CategoricalAccuracy()
)

In [ ]:
loss,accuracy = export_model.evaluate(raw_test_ds)
print(loss,accuracy)

128/128 [==============================] - 2s 16ms/step - loss: 0.5343 - categorical_accuracy: 0.8127
0.5343489646911621 0.8126562237739563


In [ ]:
inputs = """Recently, I ran some of my JavaScript code through Crockford's JSLint, and it gave the following error:

Problem at line 1 character 1: Missing "use strict" statement.
Doing some searching, I realized that some people add "use strict"; into their JavaScript code. Once I added the statement, the error stopped appearing. Unfortunately, Google did not reveal much of the history behind this string statement. Certainly it must have something to do with how the JavaScript is interpreted by the browser, but I have no idea what the effect would be.

So what is "use strict"; all about, what does it imply, and is it still relevant?

Do any of the current browsers respond to the "use strict"; string or is it for future use?"""

predictions = export_model.predict([inputs])
print(predictions,raw_train_ds.class_names[np.argmax(predictions)])

1/1 [==============================] - 0s 42ms/step
[[0.44488397 0.19873452 0.95622635 0.49443305]] javascript


In [ ]:
export_model.save_weights("./model")

In [ ]:
export_model.save("stackoverflow_multi.keras")